## Import Dependencies

In [2]:
# dependencies
from flask import Flask, render_template, jsonify
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

## Scrape the cash rate data from the RBA website and store in a Pandas Dataframe

In [4]:
# Make a request to the RBA website
url = 'https://www.rba.gov.au/statistics/cash-rate/'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the interest rate data
table = soup.find('table', {'id': 'datatable','class': 'table-linear table-numeric'})

# Initialize an empty list to store the data
data = []

# Extract the data from the table
for row in table.find_all('tr'):
    cells = row.find_all(['th','td'])
    if len(cells) == 4:
        date = cells[0].text.strip()
        change = cells[1].text.strip()
        rate = cells[2].text.strip()
        data.append([date, change, rate])
    else:
        continue
        
# Create a pandas DataFrame
rba_df = pd.DataFrame(data[1:], columns=data[0])
rba_df.columns = ["date", "change_pct", "cash_rate_pct"]

# Convert the date column to datetime then to YYYY-MM-DD format
rba_df['date'] = pd.to_datetime(rba_df['date'], format='%d %b %Y')
rba_df['date'] = rba_df['date'].dt.strftime('%Y-%m-%d')

# Filter date column to only show dates after 30th September 2011
rba_df = rba_df[rba_df['date'] > '2011-09-30']

# Print the DataFrame
print(rba_df)

           date change_pct cash_rate_pct
0    2022-12-07      +0.25          3.10
1    2022-11-02      +0.25          2.85
2    2022-10-05      +0.25          2.60
3    2022-09-07      +0.50          2.35
4    2022-08-03      +0.50          1.85
..          ...        ...           ...
120  2012-03-07       0.00          4.25
121  2012-02-08       0.00          4.25
122  2011-12-07      -0.25          4.25
123  2011-11-02      -0.25          4.50
124  2011-10-05       0.00          4.75

[125 rows x 3 columns]


## 2.	Use the SQLAlchemy library to connect to a SQL database and create a table to store the scraped data.

In [3]:
app = Flask(__name__)

# connect to database using SQLAlchemy
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'rates_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

# Confirm tables
insp.get_table_names()

# Load DataFrame into Postgres Database
df.to_sql(name='interest_rates', con=engine, if_exists='append', index=False)

# Test Database with SQL Query
pd.read_sql_query('SELECT * from interest_rates LIMIT 100', con=engine).head()

,id,date,change_pct,cash_rate_pct
0,1,2022-12-07,+0.25,3.10
1,2,2022-11-02,+0.25,2.85
2,3,2022-10-05,+0.25,2.60
3,4,2022-09-07,+0.50,2.35
4,5,2022-08-03,+0.50,1.85


In [4]:
# define a route to retrieve data from the database and serve it as a JSON response
@app.route('/api/data')
def data():
    # Query the database
    query = 'SELECT date, change_pct, cash_rate_pct FROM interest_rates'
    df = pd.read_sql_query(query, con=engine)
    # Convert the data to a JSON object
    data = df.to_json(orient='records')
    return jsonify(data)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\rober\anaconda\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
